In [1]:
import pickle
import json
import pandas as pd

In [2]:
filepath = "output/RM - HDSD ACL_KHDN_edit_final/json_response.json"
with open(filepath, "r") as f:
    data = json.load(f)

# Create DataFrame from data
df = pd.DataFrame([
    {"question": item["question"], "answer": item["answer"], "ref": item.get("ref", "")} 
    for key, item in data.items()
])
df.to_excel("output/RM - HDSD ACL_KHDN_edit_final/RM - HDSD ACL_KHDN_edit_final_QA.xlsx", index=False)

# Display the first few rows
df.head()

,question,answer,ref
0,Làm thế nào để tìm kiếm khách hàng MSME đã tồn...,- Mô tả: Cán bộ quản lý tín dụng (RM) vào menu...,"[RM - HDSD ACL_KHDN_edit_final.xlsx, Sheet 1, 1]"
1,"Khi tìm kiếm khách hàng MSME, tôi cần nhập nhữ...",- Mô tả: Cán bộ quản lý tín dụng (RM) thực hiệ...,"[RM - HDSD ACL_KHDN_edit_final.xlsx, Sheet 1, 2]"
2,"Sau khi nhấn tìm kiếm khách hàng MSME, làm sao...",- Mô tả: Cán bộ quản lý tín dụng (RM) thực hiệ...,"[RM - HDSD ACL_KHDN_edit_final.xlsx, Sheet 1, 3]"
3,Quy trình tạo mới thông tin khách hàng MSME tr...,- Mô tả: Cán bộ quản lý tín dụng (RM) vào menu...,"[RM - HDSD ACL_KHDN_edit_final.xlsx, Sheet 2, 1]"
4,"Khi tạo mới khách hàng MSME, tôi cần nhập nhữn...",- Mô tả: Cán bộ quản lý tín dụng (RM) thực hiệ...,"[RM - HDSD ACL_KHDN_edit_final.xlsx, Sheet 2, 2]"


In [3]:
with open("results.pkl", "rb") as f:
    results = pickle.load(f)


In [3]:
with open("prompts/evaluation/prompt.json", "r") as f:
    evaluate_prompts = json.load(f)

In [4]:
print(evaluate_prompts["system_prompt"])

Bạn là một chuyên gia đánh giá chất lượng câu trả lời của chatbot. Hãy đánh giá câu trả lời của agent hỗ trợ ngân hàng VIB so với câu trả lời chuẩn (true answer) dựa trên các tiêu chí sau:

## Tiêu chí đánh giá
1. Độ bao phủ thông tin (Information Coverage):
- Điểm 8-10: Câu trả lời bao gồm tất cả thông tin quan trọng có trong true answer, không bỏ sót bất kỳ thông tin nào
- Điểm 5-7: Câu trả lời bao gồm một phần thông tin quan trọng từ true answer, thiếu một số thông tin
- Điểm 0-4: Câu trả lời thiếu hầu hết thông tin quan trọng từ true answer

2. Độ chính xác và liên quan của thông tin (Information Accuracy and Relevance):
- Điểm 8-10: Thông tin bổ sung (nếu có) đều chính xác, liên quan đến câu hỏi và hỗ trợ người dùng hiểu rõ hơn
- Điểm 5-7: Thông tin bổ sung (nếu có) chính xác nhưng không hoàn toàn liên quan hoặc có thể gây hiểu nhầm nhẹ
- Điểm 0-4: Thông tin bổ sung không chính xác, không liên quan đến câu hỏi hoặc gây hiểu sai lệch nghiêm trọng

3. Định dạng và cấu trúc (Format):

In [5]:
system_prompt = """
Bạn là một chuyên gia đánh giá chất lượng câu trả lời của chatbot. Hãy đánh giá câu trả lời của agent hỗ trợ ngân hàng VIB so với câu trả lời chuẩn (true answer) dựa trên các tiêu chí sau:

## Tiêu chí đánh giá
1. Độ bao phủ thông tin (Information Coverage):
- Điểm 8-10: Câu trả lời bao gồm tất cả thông tin quan trọng có trong true answer, không bỏ sót bất kỳ thông tin nào
- Điểm 5-7: Câu trả lời bao gồm một phần thông tin quan trọng từ true answer, thiếu một số thông tin
- Điểm 0-4: Câu trả lời thiếu hầu hết thông tin quan trọng từ true answer
- Lưu ý: Tiêu chí này chỉ cần xem xét thông tin có trong true answer hay chưa, nếu thừa thì bỏ qua, không cần trừ điểm vì sẽ đánh giá ở tiêu chí khác, bỏ qua cả cách trình bày, đánh số

2. Độ chính xác và liên quan của thông tin (Information Accuracy and Relevance):
- Điểm 8-10: Thông tin bổ sung (nếu có) đều chính xác, liên quan đến câu hỏi và hỗ trợ người dùng hiểu rõ hơn
- Điểm 5-7: Thông tin bổ sung (nếu có) chính xác nhưng không hoàn toàn liên quan hoặc có thể gây hiểu nhầm nhẹ
- Điểm 0-4: Thông tin bổ sung không chính xác, không liên quan đến câu hỏi hoặc gây hiểu sai lệch nghiêm trọng

3. Định dạng và cấu trúc (Format):
- Điểm 8-10: Câu trả lời được định dạng rõ ràng, có xuống dòng khi tách ý, không có câu diễn giải hoặc hậu tố không cần thiết
- Điểm 5-7: Câu trả lời có định dạng nhưng chưa tối ưu
- Điểm 0-4: Câu trả lời không có định dạng rõ ràng

4. Ngôn ngữ và phong cách (Language):
- Điểm 8-10: Sử dụng ngôn ngữ lịch sự, chuyên nghiệp, phù hợp với ngữ cảnh ngân hàng
- Điểm 5-7: Ngôn ngữ chấp nhận được nhưng chưa thật sự chuyên nghiệp
- Điểm 0-4: Ngôn ngữ không phù hợp

5. Xử lý trường hợp không tìm thấy câu trả lời (Handling Unknown):
- Điểm 8-10: Đề xuất chuyển đến Admin một cách phù hợp khi không tìm thấy câu trả lời
- Điểm 5-7: Có đề xuất chuyển đến Admin nhưng chưa rõ ràng
- Điểm 0-4: Không xử lý trường hợp không tìm thấy câu trả lời

Hãy đánh giá câu trả lời của agent theo các tiêu chí trên và cho điểm tổng thể từ 0-10.

## Input Format:
- Câu hỏi: [câu hỏi từ người dùng]
- True Answer: [câu trả lời chuẩn từ FAQ]
- Agent Answer: [câu trả lời của agent]

## Output Format:
Đầu ra của phản hồi phải là một json schema như sau:
```json
{{
  "information_coverage": [Điểm số cho độ bao phủ thông tin (Information Coverage)],
  "hallucination_control": [Điểm số kiểm soát Hallucination],
  "format_and_structure": [Điểm số cho dạng và cấu trúc (Format and Structure)],
  "language_and_style": [Điểm số cho ngôn ngữ và phong cách (Language)],
  "handling_unknown": [Điểm số cho xử lý trường hợp không tìm thấy câu trả lời (Handling Unknown)],
  "comments": [Nhận xét, góp ý và cải thiện cho từng tiêu chí điểm]
}}
```

## Lưu ý quan trọng:
- Nếu trong câu trả lời có mô tả cho các bước, hãy bỏ qua, đây không phải điểm trừ
- Bỏ qua các câu kết thúc như "Quý khách có cần hỗ trợ gì thêm không", đây không phải là điểm trừ
- Đưa ra điểm ngang hàng với tiêu chí đó, ví dụ: Độ bao phủ thông tin (Information Coverage): 10
- Điểm tổng thể là điểm cuối cùng, không đưa ra cách tính, ví dụ: Điểm tổng thể: 9.8
- Nhận xét, góp ý và cải thiện cho từng tiêu chí điểm sẽ là json với key là tiêu chí, value là nhận xét, góp ý và cải thiện cho tiêu chí đó
""".strip()

In [6]:
evaluate_prompts["system_prompt"] = system_prompt
with open("prompts/evaluation/prompt.json", "w") as f:
    json.dump(evaluate_prompts, f, indent=4, ensure_ascii=False)


In [5]:
df = pd.read_csv("/home/vietlong/Downloads/2025-06-09T07-17_export.csv")
df = df.iloc[:, 1:]
df.head()

,Question,True Answer,Agent Answer,Chat ID,Information Coverage Score,Hallucination Score,Format Score,Language Score,Handling Unknown Score,Average Score,Comment
0,Làm thế nào xử lý khi nhập thêm giấy tờ tùy th...,1. HƯớng dẫn trên màn hình Dùng Menu Thông tin...,Khi nhập thêm giấy tờ tùy thân của khách hàng ...,cf02507d-3c84-4809-ac87-a5de86bee3dc,10,10,10,10,10,10.0,Agent đã cung cấp đầy đủ thông tin từ câu trả ...
1,Quy trình xử lý khi chọn giấy tờ tùy thân làm ...,1. HƯớng dẫn trên màn hình Dùng Menu Thông tin...,Khi chọn giấy tờ tùy thân làm giấy tờ chính nh...,65f532e1-f96c-4fc3-832f-9b1dcb30870d,10,10,10,10,10,10.0,Agent đã cung cấp đầy đủ thông tin từ câu trả ...
2,Cần làm gì khi phải bổ sung giấy tờ tùy thân b...,1. HƯớng dẫn trên màn hình Dùng Menu Thông tin...,Khi cần bổ sung giấy tờ tùy thân bị thiếu tron...,b13b20a0-f380-4f4b-a970-fae2cd7ada5a,9,8,10,10,10,9.4,Agent đã bao phủ hầu hết thông tin quan trọng ...
3,Làm sao xử lý khi hệ thống báo lỗi Client no k...,Kiểm tra bằng email hoặc nhờ quầy kiểm tra trê...,Để xử lý khi hệ thống báo lỗi Client no không ...,58082da5-5d52-4faa-900c-665e52496293,10,10,10,10,10,10.0,"Câu trả lời của agent rất tốt, bao gồm đầy đủ ..."
4,Quy trình xử lý khi hệ thống báo lỗi sai ngày ...,Kiểm tra thông tin ngày sinh trên ACL trên hồ ...,Khi hệ thống báo lỗi sai ngày sinh và dừng luồ...,2d2c0d19-7fa7-4dbf-8b94-370f976a3787,10,10,10,10,10,10.0,Agent đã cung cấp đầy đủ thông tin từ câu trả ...


In [7]:
df[df['Information Coverage Score'] <= 8].to_csv("filtered_autotest_results.csv", index=False)

In [10]:
import numpy as np

df['Average Score'].sum()/df.shape[0]

9.450000000000001

In [14]:
df.to_csv("/home/vietlong/Downloads/qa_results.csv", encoding="utf-8-sig", index=False)

In [1]:
print("""
\nCâu hỏi (Question): \n```\n{question}\n```\n\nCâu trả lời chuẩn (True Answer): \n```\n{true_answer}\n```\n\nCâu trả lời của agent (Agent Answer): \n```\n{agent_answer}\n```\n
      """.strip())

Câu hỏi (Question): 
```
{question}
```

Câu trả lời chuẩn (True Answer): 
```
{true_answer}
```

Câu trả lời của agent (Agent Answer): 
```
{agent_answer}
```
